In [5]:
import pandas as pd
import os
import json
dataset_list = ['banking', 'stackoverflow', 'clinc', 'hwu', 'mcid', 'thucnews', 'ele']
# dataset_list = ['news']
rates = [0.25, 0.5, 0.75]


In [6]:
data_statics = {}
for dataset_name in dataset_list:
    ans = {}
    data_statics[dataset_name] = {}
    for split in ['train', 'dev', 'test']:
        data_statics[dataset_name][split] = {}
        df = pd.read_csv(f'{dataset_name}/origin_data/{split}.tsv', sep='\t')
        df.columns = ['text', 'label']
        ans[split] = df
    df = pd.concat([ans[i] for i in ans])
    os.makedirs(f'{dataset_name}/label', exist_ok=True)
    pd.DataFrame(df['label'].unique()).to_csv(f'{dataset_name}/label/label.list', index=None, header=None)

In [7]:
import pandas as pd
import os
import numpy as np

fold_settings = [5, 10]

def shuffle(shuffled_classes, fold_num, known_class_num, step=None):
    total_class_num = len(shuffled_classes)
    if step is None:
        step = max(1, total_class_num // fold_num)  # 默认步长为一半，避免完全重叠

    selected_class_groups = []

    for i in range(fold_num):
        start = (i * step) % total_class_num
        indices = [(start + j) % total_class_num for j in range(known_class_num)]
        fold_classes = [shuffled_classes[idx] for idx in indices]
        selected_class_groups.append(fold_classes)

    return selected_class_groups

for dataset_name in dataset_list:
    label_path = f'{dataset_name}/label/label.list'
    label_list = pd.read_csv(label_path, header=None)[0]  # 假设每行是一个label
    total_class_num = len(label_list)

    for fold_num in fold_settings:
        for rate in rates:
            known_class_num = round(rate * total_class_num)
            rng = np.random.default_rng(seed=42)
            selected_class_groups = shuffle(label_list, fold_num, known_class_num)

            for fold_idx, fold_classes in enumerate(selected_class_groups):
                fold_dir = f'{dataset_name}/label/fold{fold_num}/part{fold_idx}'
                os.makedirs(fold_dir, exist_ok=True)
                output_path = os.path.join(fold_dir, f'label_known_{rate}.list')
                pd.DataFrame(fold_classes).to_csv(output_path, index=False, header=False)

In [8]:
import pandas as pd
import os
import json
import random
for dataset_name in dataset_list:
    for mode in ['train', 'dev']:
        df = pd.read_csv(f'{dataset_name}/origin_data/{mode}.tsv', sep='\t')
        for labeled_ratio in [0.1, 0.5, 1.0]:   
            df_group = df.groupby('label').agg(list)
            df_group['text'] = df_group['text'].apply(lambda x: random.sample(x, max(round(len(x) * labeled_ratio), 1)))
            df_group = df_group.reset_index().explode('text')[['text', 'label']]
            os.makedirs(f'{dataset_name}/labeled_data/{labeled_ratio}', exist_ok=True)
            known_text_list = df_group['text'].tolist()
            sub_df = df[:]
            sub_df['labeled'] = sub_df['text'].apply(lambda x: x in known_text_list)
            sub_df = sub_df.reset_index()[['label', 'labeled']]
            sub_df.to_csv(f'{dataset_name}/labeled_data/{labeled_ratio}/{mode}.tsv', sep='\t', index=None)

/tmp/ipykernel_2360868/140616066.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['labeled'] = sub_df['text'].apply(lambda x: x in known_text_list)
/tmp/ipykernel_2360868/140616066.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['labeled'] = sub_df['text'].apply(lambda x: x in known_text_list)
/tmp/ipykernel_2360868/140616066.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t